## Compute revelance for neural network that uses tabular data

Here we demonstrate how we can calculate the layer wise relevence of a neural network which can provide insights on important features that contribute to the target variable 

In [ ]:
#Imports
import torch
import torch.nn as nn
#import torchvision
#import torchvision.models as models
#import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
import pandas as pd 
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torch.optim import Adam
from IPython import display
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
img_dat_nc=pd.read_csv("/Users/seymour/Desktop/SuStaIn_15_June/Andre_sent_21_July/img_dat_nc.csv")
img_dat_nc["SubjID"] = img_dat_nc["Site"] + img_dat_nc["SubjID"].astype(str)+ img_dat_nc["SDx_dti"].astype(str)
img_dat_nc["SubjID"] = img_dat_nc["SubjID"].str[:-2]
img_dat_nc["SDx_dti"] = img_dat_nc["SDx_dti"].map({3: 0, 4: 1})

In [ ]:
t1_feats=['Lhippo','Rhippo','Lamyg','Ramyg','LLatVent','RLatVent','Lthal','Rthal','Lcaud','Rcaud','Lput','Rput','Lpal','Rpal','Laccumb','Raccumb','L_frontalL_volume','L_parietalL_volume','L_temporalL_volume','L_occipitalL_volume','L_cingulateC_volume','R_frontalL_volume','R_parietalL_volume','R_temporalL_volume','R_occipitalL_volume','R_cingulateC_volume']

In [ ]:
X=img_dat_nc[t1_feats]
X_normalized=normalize(X,axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, img_dat_nc.SDx_dti, test_size=0.3,random_state=109) # 70% training and 30% test

In [ ]:
#Change data type to match pytorch tensor format
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

input_size = X_train.shape[1]
hidden_sizes = [50, 64]
output_size = 2

# Define the model


model = nn.Sequential(OrderedDict([
                  ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                  ('relu1', nn.ReLU()),
                  ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                  ('relu2', nn.ReLU()),
                  ('fc_output', nn.Linear(hidden_sizes[1], output_size)),
                  ('softmax', nn.Softmax(dim=1))]))

# Define the optimizer and loss function
optimizer = Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


y_train = torch.from_numpy(y_train.values)
y_train = y_train.long()

num_epochs=10
# Train the model
for epoch in range(num_epochs):
    
    output = model(X_train)
    output = output.float()
    loss = criterion(output, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Make predictions on the test set
with torch.no_grad():
    test_output = model(X_test)
    test_predictions = torch.sigmoid(test_output)
    #test_predictions=test_predictions.round()

# Calculate AUC metric
auc = roc_auc_score(y_test, test_predictions[:,0])
print("AUC: ", auc)


In [ ]:
accuracy_score(y_test, test_predictions[:,0].round())

In [ ]:
#aLminus1=activation of layer l-1
#Rj= last R_nextk

def lrp(model, x,pred):
    model.eval()
    xj=x
    namej="fc1"
    length=len(list(model.modules()))
    print(length)
    
    for i, (name, layer) in enumerate(model._modules.items()):
        print(f"{name}")

#         print(f"Layer {i+1}:{name}")
#         print(layer)
        
        if i==0:
            out = model._modules[f"{name}"](x)
            w = model.state_dict()[f"{name}.weight"]
            Rk = out.data
            aLminus1 = x.data
            z = aLminus1.unsqueeze(0).numpy().dot(w.T.numpy()) #Here summation of aj.wk is done via matrix dot product
            s = Rk/z
            c = s.numpy().dot(w.numpy())
            R_nextk=c

               
            
        if i>0:
            print(type(model._modules[name]))

            if isinstance(model._modules[name], torch.nn.Conv2d) or\
            isinstance(model._modules[name], torch.nn.AvgPool2d) or\
            isinstance(model._modules[name], torch.nn.Linear):
                print(f"{name}")
                
                if name=='fc_output':
                    outj = pred
                else:outj = model._modules[f"{name}"](out)
   
                
                wj = model.state_dict()[f"{name}.weight"]
                Rk = R_nextk
                aLminus1 = out.data #out is the previous layer activation
                z = aLminus1.unsqueeze(0).numpy().dot(wj.T.numpy())
                z=z.T
                s = Rk/z
                cj = s.T.dot(wj.numpy()) 
                sum_cj=np.sum(cj,axis=1) 

                #update parameters for next iteration
                out=outj
                R_nextk=sum_cj
                
    return R_nextk,out

relevance,pred1=lrp(model, X_test[0,:],test_predictions)
relevance